In [2]:
# Get hidden states by context level and by layer

In [3]:
import os
os.environ['HF_HOME'] = '/sbksvol/amurali/'

from huggingface_hub import login
login(token = "<hf_token>")

import transformers
import torch

import csv
import numpy as np
import pandas as pd

from transformers import AutoConfig, AutoModel, AutoTokenizer, AutoModelForCausalLM, pipeline, LlamaForCausalLM

from tqdm import tqdm

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /sbksvol/amurali/token
Login successful


In [4]:
print(f"PyTorch Version: {torch.__version__}")
print(f"Transformers Version: {transformers.__version__}")
print(f"NumPy Version: {np.__version__}")

PyTorch Version: 1.13.1+cu117
Transformers Version: 4.30.2
NumPy Version: 1.21.2


In [4]:
# Load model/create pipeline
model_name = "ivnle/llamatales_jr_8b-lay8-hs512-hd8-33M"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, output_hidden_states=True).to("cuda")
# llamatales_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)

# Prompts
prompts = {
    1: "Once upon a time there was a dragon", 
    2: "Once upon a time there was a princess", 
    3: "Once upon a time there were two children",
    4: "Once upon a time there was a prince",
    5: "Once upon a time there was a frog",
    6: "Once upon a time there was a king",
    7: "Once upon a time there was a queen",
    8: "Once upon a time there was a wolf",
    9: "Once upon a time there was a genie",
    10: "Once upon a time there was a poor boy"
}


for prompt_id, prompt_text in prompts.items():
    print(f"Prompt {prompt_id}: \"{prompt_text}\"")
    data = []
    npz_data = {}
    hidden_state_file = f'./hidden_states/prompt_{prompt_id}.npz'
    for i in tqdm(range(1000)):
        # Generate using pipeline
        # sequences = llamatales_pipeline(
        #     prompt_text,
        #     do_sample=True,
        #     top_k=10,
        #     num_return_sequences=1,
        #     max_new_tokens=100
        # )
        # generated_story = sequences[0]['generated_text']
#         print(f"\nGenerated (pipeline):\n{generated_story}")

        # Count tokens
        # num_tokens_generated_story = len(tokenizer.encode(generated_story))

        # Generate hidden states from model.generate using same prompt
        inputs = tokenizer(prompt_text, return_tensors="pt").to("cuda")
        outputs = model.generate(
            inputs.input_ids,
            attention_mask=inputs.attention_mask,
            do_sample=True,
            top_k=10,
            num_return_sequences=1,
            max_new_tokens=512,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id,
            return_dict_in_generate=True,
            output_hidden_states=True
        )
        
        generated_story = tokenizer.batch_decode(outputs[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)[0]
#         print("New Story: \n", new_story)
        
#         print("Length of Generated Story: ", num_tokens_generated_story)
#         print("Length of New Story: ", len(tokenizer.encode(new_story)))
        
        # Map output tokens to prompt ID
        # output_id = [prompt_id] * num_tokens_generated_story

        # Extract hidden states
        convert_hidden_states = []
#         print(f"Type of hidden_states: {type(outputs.hidden_states)}")
        for layer_hidden_states in outputs.hidden_states:
            convert_hidden_states.append([t.detach().cpu().numpy() for t in layer_hidden_states])

        # Debug hidden state shape
#         print(f"Hidden states shape: {len(convert_hidden_states)} generated tokens × "
#               f"{len(convert_hidden_states[0])} layers × "
#               f"{len(convert_hidden_states[0][0])} sequences × "
#               f"{len(convert_hidden_states[0][0][0])} tokens × " 
#               f"{len(convert_hidden_states[0][0][0][0])} dims")

#         print("Num Tokens: ", len(convert_hidden_states[0][0][0])) #same as context (length of original generated story)

        # Save hidden states to file
        arr = np.empty(len(convert_hidden_states), dtype=object)
        arr[:] = convert_hidden_states
        npz_data[f"arr_{i}"] = arr
        # npz_data[f"arr_{i}"] = convert_hidden_states
        
        num_tokens_generated_story = len(tokenizer.encode(generated_story))
        
        # Store results
        data.append([prompt_id, prompt_text, generated_story, hidden_state_file, num_tokens_generated_story])
    
    np.savez_compressed(hidden_state_file, **npz_data)
    
    df = pd.DataFrame(data, columns=["prompt_id", "prompt", "story", "hidden_state_file", "len_generated_story"])
    if(prompt_id == 1): df.to_csv("story_dataset_new.csv", index = False)
    else:
        df.to_csv('story_dataset_new.csv', mode='a', header = False, index = False)
        


Some weights of LlamaForCausalLM were not initialized from the model checkpoint at ivnle/llamatales_jr_8b-lay8-hs512-hd8-33M and are newly initialized: ['model.layers.7.self_attn.rotary_emb.inv_freq', 'model.layers.4.self_attn.rotary_emb.inv_freq', 'model.layers.0.self_attn.rotary_emb.inv_freq', 'model.layers.2.self_attn.rotary_emb.inv_freq', 'model.layers.5.self_attn.rotary_emb.inv_freq', 'model.layers.6.self_attn.rotary_emb.inv_freq', 'model.layers.1.self_attn.rotary_emb.inv_freq', 'model.layers.3.self_attn.rotary_emb.inv_freq']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Prompt 1: "Once upon a time there was a dragon"


  0%|                                                                                                 | 0/1000 [00:00<?, ?it/s]/opt/conda/lib/python3.7/site-packages/transformers/generation/utils.py:1260: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  "You have modified the pretrained model configuration to control generation. This is a"
  1%|▊                                                                                        | 9/1000 [00:23<43:19,  2.62s/it]
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3444, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_330/2441109779.py", line 54, in <module>
    output_hidden_states=True
  File "/opt/conda/lib/python3.7/site-packages/torch/autograd/grad_mode.py", line 27, in decorate_context
    return func(*args, **kwargs)
  File "/opt/conda/lib/python3.7/site-packages/transformers/generation/utils.py", line 1583, in generate
    **model_kwargs,
  File "/opt/conda/lib/python3.7/site-packages/transformers/generation/utils.py", line 2619, in sample
    outputs = self(
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2064, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attri

TypeError: object of type 'NoneType' has no len()

In [5]:
# Load model/create pipeline
model_name = "ivnle/llamatales_jr_8b-lay8-hs512-hd8-33M"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, output_hidden_states=True).to("cuda")

# Prompts
prompts = {
    1: "Once upon a time there was a dragon", 
    2: "Once upon a time there was a princess", 
    3: "Once upon a time there were two children",
    4: "Once upon a time there was a prince",
    5: "Once upon a time there was a frog",
    6: "Once upon a time there was a king",
    7: "Once upon a time there was a queen",
    8: "Once upon a time there was a wolf",
    9: "Once upon a time there was a genie",
    10: "Once upon a time there was a poor boy"
}

Some weights of LlamaForCausalLM were not initialized from the model checkpoint at ivnle/llamatales_jr_8b-lay8-hs512-hd8-33M and are newly initialized: ['model.layers.7.self_attn.rotary_emb.inv_freq', 'model.layers.6.self_attn.rotary_emb.inv_freq', 'model.layers.1.self_attn.rotary_emb.inv_freq', 'model.layers.2.self_attn.rotary_emb.inv_freq', 'model.layers.5.self_attn.rotary_emb.inv_freq', 'model.layers.4.self_attn.rotary_emb.inv_freq', 'model.layers.3.self_attn.rotary_emb.inv_freq', 'model.layers.0.self_attn.rotary_emb.inv_freq']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
for prompt in prompts.values():
    print(len(tokenizer.encode(prompt)))

9
9
9
9
9
9
9
9
9
10


In [ ]:
# for i in range(1000):
#     if(len(npz_data[f"arr_{i}"][0][0][0][0]) != 512):
#         # print(i)
#         print(len(npz_data[f"arr_{i}"]))

In [ ]:
# npz_data["arr_1"][0][0].shape

In [ ]:
#100x9x1x9x512

In [ ]:
df = pd.read_csv("story_dataset_new.csv")
df